In [1]:
import sys
import time
import torch
import torch.nn as nn
import torch.nn.functional as F

# 기본 라이브러리
import os
import json
import shutil

# PyTorch 관련 라이브러리
import torch.utils.data as data
import torch.backends.cudnn as cudnn

# 이미지 변환 관련 라이브러리
from torchvision import transforms

# 데이터 로더 및 모델, 유틸리티 함수 불러오기
from data_loader import get_segmentation_dataset  # 데이터셋 불러오는 함수
from utils.loss import MixSoftmaxCrossEntropyLoss, MixSoftmaxCrossEntropyOHEMLoss  # 손실 함수
from utils.lr_scheduler import LRScheduler  # 학습률 스케줄러
from utils.metric import SegmentationMetric  # 평가 지표

from modules import conv3x3, conv1x1, DWConvBNAct, PWConvBNAct, ConvBNAct, Activation, SegHead
from model_registry import register_model, aux_models

# Python 모듈 경로에 'models' 폴더 추가
models_path = "/home/segmentsafestep/Fast-SCNN-pytorch-master/models"

if models_path not in sys.path:
    sys.path.append(models_path)  # models 폴더 추가
    
# sys.path 확인 (제대로 추가되었는지)
print("Python 경로 목록:")
print("\n".join(sys.path))

train_img_path = "/home/segmentsafestep/Fast-SCNN-pytorch-master/datasets/indobohaeng/leftImg8bit/train"
dataset_root = "/home/segmentsafestep/Fast-SCNN-pytorch-master/datasets"

Python 경로 목록:
/home/segmentsafestep/miniconda3/envs/jupyter-env/lib/python39.zip
/home/segmentsafestep/miniconda3/envs/jupyter-env/lib/python3.9
/home/segmentsafestep/miniconda3/envs/jupyter-env/lib/python3.9/lib-dynload

/home/segmentsafestep/miniconda3/envs/jupyter-env/lib/python3.9/site-packages
/home/segmentsafestep/miniconda3/envs/jupyter-env/lib/python3.9/site-packages/IPython/extensions
/home/segmentsafestep/.ipython
/home/segmentsafestep/Fast-SCNN-pytorch-master/models


In [2]:
import os
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm

# 🎨 원본 RGB → 클래스 ID 매핑
rgb_to_class = {
    (208, 88, 255): 1, (88, 38, 128): 3, (230, 170, 255): 1, (138, 60, 200): 1,
    (255, 128, 255): 1, (255, 155, 155): 3, (128, 96, 0): 3, (255, 255, 0): 3,
    (255, 128, 0): 3, (255, 0, 0): 0, (105, 105, 255): 3, (255, 192, 0): 0,
    (0, 255, 0): 0, (255, 0, 255): 1, (128, 128, 128): 2, (0, 0, 255): 2,
    (217, 217, 217): 2, (55, 86, 35): 3, (110, 168, 70): 2, (255, 230, 153): 2,
    (198, 89, 17): 2, (0, 0, 0): 3
}

# 클래스 → 색상 (컬러용)
color_map = {
    0: (255, 0, 0),      # 빨강
    1: (255, 255, 0),    # 노랑
    2: (0, 255, 0),      # 초록
    3: (0, 0, 0)         # 검정 (background)
}

# 클래스 → 밝기 (흑백용)
brightness_map = {
    0: 200,  # caution zone (주의 구간)
    1: 150,  # roadway + alley
    2: 100,  # sidewalk (인도)
    3: 50    # background (배경)
}

# 🔍 소문자 파일만 판별
def is_lower_filename(filename):
    name_only = os.path.splitext(os.path.basename(filename))[0]
    letters = [c for c in name_only if c.isalpha()]
    return letters and all(c.islower() for c in letters)

# RGB → 클래스 맵핑
def rgb_to_class_map(image_np):
    h, w, _ = image_np.shape
    class_map = np.full((h, w), 3, dtype=np.uint8)  # 기본값 background

    for rgb, class_id in rgb_to_class.items():
        mask = np.all(image_np == np.array(rgb), axis=-1)
        class_map[mask] = class_id

    return class_map

# 클래스 → RGB 컬러 맵핑
def class_to_rgb(mask_2d):
    h, w = mask_2d.shape
    rgb = np.zeros((h, w, 3), dtype=np.uint8)

    for class_id, color in color_map.items():
        mask = (mask_2d == class_id)
        rgb[..., 0][mask] = color[0]
        rgb[..., 1][mask] = color[1]
        rgb[..., 2][mask] = color[2]

    return rgb

# 클래스 → 흑백 맵핑 (밝기 값)
def class_to_gray(mask_2d):
    gray = np.zeros(mask_2d.shape, dtype=np.uint8)

    for class_id, brightness in brightness_map.items():
        mask = (mask_2d == class_id)
        gray[mask] = brightness

    return gray

# 메인 변환 함수 (🔥 전체 이미지 변환)
def process_images(input_dir, output_color_dir, output_gray_dir):
    print(f"\n🚀 변환 시작: {input_dir}")

    image_paths = glob.glob(os.path.join(input_dir, "*.png"))
    total_images = len(image_paths)
    print(f"총 {total_images}개의 이미지 파일을 찾았습니다.")

    # 🔤 소문자 파일만 선택
    lower_files = [f for f in image_paths if is_lower_filename(f)]
    print(f"🔤 소문자 이미지: {len(lower_files)}개")

    # 저장 폴더 생성
    os.makedirs(output_color_dir, exist_ok=True)
    os.makedirs(output_gray_dir, exist_ok=True)

    for in_path in tqdm(lower_files, desc="🎨 컬러 + 흑백 변환 중"):
        image = Image.open(in_path).convert("RGB")
        image_np = np.array(image)

        class_map = rgb_to_class_map(image_np)

        # 🎨 컬러 저장
        rgb_label = class_to_rgb(class_map)
        out_color_path = os.path.join(output_color_dir, os.path.basename(in_path))
        Image.fromarray(rgb_label).save(out_color_path)

        # 🖤 흑백 저장 (파일명은 대문자)
        gray_label = class_to_gray(class_map)
        base_name = os.path.splitext(os.path.basename(in_path))[0]
        upper_name = base_name.upper() + ".png"
        out_gray_path = os.path.join(output_gray_dir, upper_name)
        Image.fromarray(gray_label).save(out_gray_path)

    print(f"\n✅ 변환 완료: 총 {len(lower_files)}개 파일 저장 완료!")

# 🔥 사용 예시
input_dir = "/home/segmentsafestep/Fast-SCNN-pytorch-master/datasets/indobohaeng/gtFine_5class/Val/alley"
output_color_dir = "/home/segmentsafestep/Fast-SCNN-pytorch-master/datasets/indobohaeng/gtFine/Val/alley"
output_gray_dir = "/home/segmentsafestep/Fast-SCNN-pytorch-master/datasets/indobohaeng/gtFine/Val/alley"

process_images(input_dir, output_color_dir, output_gray_dir)


🚀 변환 시작: /home/segmentsafestep/Fast-SCNN-pytorch-master/datasets/indobohaeng/gtFine_5class/Val/alley
총 2306개의 이미지 파일을 찾았습니다.
🔤 소문자 이미지: 2306개


🎨 컬러 + 흑백 변환 중: 100%|███████████████████████████████████████████████████| 2306/2306 [59:24<00:00,  1.55s/it]


✅ 변환 완료: 총 2306개 파일 저장 완료!


In [3]:
import os
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm

# 🎨 원본 RGB → 클래스 ID 매핑
rgb_to_class = {
    (208, 88, 255): 1, (88, 38, 128): 3, (230, 170, 255): 1, (138, 60, 200): 1,
    (255, 128, 255): 1, (255, 155, 155): 3, (128, 96, 0): 3, (255, 255, 0): 3,
    (255, 128, 0): 3, (255, 0, 0): 0, (105, 105, 255): 3, (255, 192, 0): 0,
    (0, 255, 0): 0, (255, 0, 255): 1, (128, 128, 128): 2, (0, 0, 255): 2,
    (217, 217, 217): 2, (55, 86, 35): 3, (110, 168, 70): 2, (255, 230, 153): 2,
    (198, 89, 17): 2, (0, 0, 0): 3
}

# 클래스 → 색상 (컬러용)
color_map = {
    0: (255, 0, 0),      # 빨강
    1: (255, 255, 0),    # 노랑
    2: (0, 255, 0),      # 초록
    3: (0, 0, 0)         # 검정 (background)
}

# 클래스 → 밝기 (흑백용)
brightness_map = {
    0: 200,  # caution zone (주의 구간)
    1: 150,  # roadway + alley
    2: 100,  # sidewalk (인도)
    3: 50    # background (배경)
}

# 🔍 소문자 파일만 판별
def is_lower_filename(filename):
    name_only = os.path.splitext(os.path.basename(filename))[0]
    letters = [c for c in name_only if c.isalpha()]
    return letters and all(c.islower() for c in letters)

# RGB → 클래스 맵핑
def rgb_to_class_map(image_np):
    h, w, _ = image_np.shape
    class_map = np.full((h, w), 3, dtype=np.uint8)  # 기본값 background

    for rgb, class_id in rgb_to_class.items():
        mask = np.all(image_np == np.array(rgb), axis=-1)
        class_map[mask] = class_id

    return class_map

# 클래스 → RGB 컬러 맵핑
def class_to_rgb(mask_2d):
    h, w = mask_2d.shape
    rgb = np.zeros((h, w, 3), dtype=np.uint8)

    for class_id, color in color_map.items():
        mask = (mask_2d == class_id)
        rgb[..., 0][mask] = color[0]
        rgb[..., 1][mask] = color[1]
        rgb[..., 2][mask] = color[2]

    return rgb

# 클래스 → 흑백 맵핑 (밝기 값)
def class_to_gray(mask_2d):
    gray = np.zeros(mask_2d.shape, dtype=np.uint8)

    for class_id, brightness in brightness_map.items():
        mask = (mask_2d == class_id)
        gray[mask] = brightness

    return gray

# 메인 변환 함수 (🔥 전체 이미지 변환)
def process_images(input_dir, output_color_dir, output_gray_dir):
    print(f"\n🚀 변환 시작: {input_dir}")

    image_paths = glob.glob(os.path.join(input_dir, "*.png"))
    total_images = len(image_paths)
    print(f"총 {total_images}개의 이미지 파일을 찾았습니다.")

    # 🔤 소문자 파일만 선택
    lower_files = [f for f in image_paths if is_lower_filename(f)]
    print(f"🔤 소문자 이미지: {len(lower_files)}개")

    # 저장 폴더 생성
    os.makedirs(output_color_dir, exist_ok=True)
    os.makedirs(output_gray_dir, exist_ok=True)

    for in_path in tqdm(lower_files, desc="🎨 컬러 + 흑백 변환 중"):
        image = Image.open(in_path).convert("RGB")
        image_np = np.array(image)

        class_map = rgb_to_class_map(image_np)

        # 🎨 컬러 저장
        rgb_label = class_to_rgb(class_map)
        out_color_path = os.path.join(output_color_dir, os.path.basename(in_path))
        Image.fromarray(rgb_label).save(out_color_path)

        # 🖤 흑백 저장 (파일명은 대문자)
        gray_label = class_to_gray(class_map)
        base_name = os.path.splitext(os.path.basename(in_path))[0]
        upper_name = base_name.upper() + ".png"
        out_gray_path = os.path.join(output_gray_dir, upper_name)
        Image.fromarray(gray_label).save(out_gray_path)

    print(f"\n✅ 변환 완료: 총 {len(lower_files)}개 파일 저장 완료!")

# 🔥 사용 예시
input_dir = "/home/segmentsafestep/Fast-SCNN-pytorch-master/datasets/indobohaeng/gtFine_5class/Train/alley"
output_color_dir = "/home/segmentsafestep/Fast-SCNN-pytorch-master/datasets/indobohaeng/gtFine/Train/alley"
output_gray_dir = "/home/segmentsafestep/Fast-SCNN-pytorch-master/datasets/indobohaeng/gtFine/Train/alley"

process_images(input_dir, output_color_dir, output_gray_dir)


🚀 변환 시작: /home/segmentsafestep/Fast-SCNN-pytorch-master/datasets/indobohaeng/gtFine_5class/Train/alley
총 18451개의 이미지 파일을 찾았습니다.
🔤 소문자 이미지: 18451개


🎨 컬러 + 흑백 변환 중:   0%|                                                   | 3/18451 [00:05<9:06:58,  1.78s/it]


KeyboardInterrupt: 